In [1]:
import pandas as pd

In [2]:
from preprocessing import preprocess

In [5]:
dfs = []
dfs.append(preprocess('../data/raw/CA/CA_1981-1985.txt'))
dfs.append(preprocess('../data/raw/CA/CA_1985-1989.txt'))
dfs.append(preprocess('../data/raw/CA/CA_1989-1993.txt'))
dfs.append(preprocess('../data/raw/CA/CA_1993-1997.txt'))
dfs.append(preprocess('../data/raw/CA/CA_1997-2001.txt'))
dfs.append(preprocess('../data/raw/CA/CA_2001-2005.txt'))
dfs.append(preprocess('../data/raw/CA/CA_2005-2009.txt'))
dfs.append(preprocess('../data/raw/CA/CA_2009-2015.txt'))

In [6]:
df = pd.concat(dfs)

In [229]:
df.columns

Index(['STN', 'WBAN', 'YEARMODA', 'TEMP', 'TEMP_count', 'DEWP', 'DEWP_count',
       'SLP', 'SLP_count', 'STP', 'STP_count', 'VISIB', 'VISIB_count', 'WDSP',
       'WDSP_count', 'MXSPD', 'GUST', 'MAX', 'MIN', 'PRCP', 'SNDP', 'FRSHTT',
       'MAX_flag', 'MIN_flag', 'PRCP_A', 'PRCP_B', 'PRCP_C', 'PRCP_D',
       'PRCP_G', 'PRCP_H', 'PRCP_I', 'DIFF'],
      dtype='object')

In [230]:
df['DIFF'] = df['MAX'] - df['MIN']
df_subsetted = df[['STN','YEARMODA', 'TEMP', 'PRCP', 'GUST', 'WDSP', 'MXSPD']]
# df_subsetted = df_subsetted.set_index(['YEARMODA', 'STN'])
df_subsetted.head()

,STN,YEARMODA,TEMP,PRCP,GUST,WDSP,MXSPD
0,691414,1985-04-24,54.1,0,15.0,5.6,9.9
1,691414,1985-04-25,42.9,NaN,25.1,12.6,19.0
2,691414,1985-04-26,43.0,NaN,20.0,8.8,15.9
3,691414,1985-04-27,42.4,NaN,53.0,16.3,29.9
4,691414,1985-04-28,40.5,NaN,40.0,14.0,27.0


In [231]:
df_subsetted.to_csv('../data/preprocessed/CA/CA_1981-2015_subsetted2.csv')

In [15]:
df_subsetted = pd.read_csv('../data/preprocessed/CA/CA_1981-2015_subsetted2.csv', parse_dates=['YEARMODA'])

##Seperate Novembers

In [188]:
df_monthly = df_subsetted.groupby('STN').apply(lambda x: x.set_index('YEARMODA')[['TEMP', 'DIFF', 'PRCP', 'GUST']].resample('M'))
df_monthly = df_monthly.reset_index()

In [189]:
def extract_novembers(df):
    df_timed = df.set_index('YEARMODA')[['TEMP', 'DIFF', 'PRCP', 'GUST']]
    return df_timed[df_timed.index.month == 11]

In [190]:
df_novembers = df_monthly.groupby('STN').apply(extract_novembers)
df_novembers = df_novembers.reset_index()

In [192]:
df_novembers.to_csv('../data/preprocessed/CA/CA_1981-2015_novembers.csv')

In [193]:
df_novembers = pd.read_csv('../data/preprocessed/CA/CA_1981-2015_novembers.csv', parse_dates=['YEARMODA'])
df_novembers = df_novembers[['STN', 'YEARMODA', 'TEMP', 'DIFF', 'PRCP', 'GUST']]

In [195]:
dff = df_novembers.set_index('YEARMODA')['1990':'2014']
dff = dff.reset_index()

In [196]:
all_data = dff.pivot_table(index=['YEARMODA'], columns=['STN']).dropna(axis=1)

In [197]:
all_data

TEMP                                                         \
STN            690150     722810     722860     722868     722880     722885   
YEARMODA                                                                       
1990-11-30  61.416667  64.792593  60.163333  69.406667  61.866667  65.836667   
1991-11-30  60.700000  66.615385  61.146667  68.740000  62.093333  61.856667   
1992-11-30  60.042857  64.566667  58.073333  63.406667  61.480000  68.363333   
1993-11-30  55.627586  65.636000  58.690323  63.116129  60.335484  64.777419   
1994-11-30  57.335294  60.837500  51.390000  60.996667  54.340000  59.603333   
1995-11-30  63.900000  71.191304  59.976667  73.856667  61.996552  62.825000   
1996-11-30  59.940741  68.848000  58.063333  69.460000  61.270000  65.063333   
1997-11-30  58.816129  69.291667  58.629032  67.745161  61.806452  65.435484   
1998-11-30  56.592000  65.652174  56.916667  64.506667  57.100000  61.323333   
1999-11-30  60.176667  71.017391  58.636667  67.786667  59.886667  64.640000   
2000-11-30  51.123333  61.785714  53.080000  59.943333  56.220000  56.953333   
2001-11-30  60.180645  69.939130  57.735484  68.651613  59.458065  59.264516   
2002-11-30  59.479310  69.285714  61.810000  68.423333  63.113333  63.575862   
2003-11-30  52.850000  64.142857  55.476667  60.840000  56.713333  58.963333   
2004-11-30  58.086667  63.421739  53.126667  60.620000  56.403333  57.883333   
2005-11-30  64.132258  68.323077  61.135484  66.977419  61.996774  63.058065   
2006-11-30  58.433333  69.339130  59.713333  68.343333  61.970000  61.806667   
2007-11-30  59.906667  65.416667  57.696667  69.463333  60.070000  59.450000   
2008-11-30  60.731034  67.246667  60.143333  70.110000  63.516667  62.906667   
2009-11-30  57.683871  64.135484  57.077419  67.374194  61.203226  60.570968   
2010-11-30  54.280000  62.396667  54.266667  64.133333  59.166667  59.723333   
2011-11-30  53.096429  66.277778  53.293333  61.610000  57.053333  58.620000   
2012-11-30  58.493333  65.370000  57.180000  67.770000  61.130000  60.863333   
2013-11-30  56.370000  63.210000  57.060000  65.483333  63.746667  62.336667   
2014-11-30  58.506667  64.531034  59.096667  66.460000  63.616667  65.143333   

                                                          ...           GUST  \
STN            722886     722897     722900     722903    ...         725905   
YEARMODA                                                  ...                  
1990-11-30  63.303333  57.713333  62.500000  65.016667    ...      20.000000   
1991-11-30  66.320000  59.946667  61.586667  65.290000    ...      21.000000   
1992-11-30  65.973333  59.176667  62.580000  65.170000    ...      15.300000   
1993-11-30  62.548387  57.577419  61.348387  63.590323    ...      19.760000   
1994-11-30  57.863333  52.233333  56.660000  58.376667    ...      19.981818   
1995-11-30  65.078571  62.427586  62.450000  64.207143    ...      14.950000   
1996-11-30  63.256667  59.680000  61.260000  63.636667    ...      19.650000   
1997-11-30  66.348387  62.251613  64.151613  66.700000    ...      22.640000   
1998-11-30  57.860000  53.436667  59.533333  57.813333    ...      21.077778   
1999-11-30  60.116667  56.066667  60.366667  58.523333    ...      19.981818   
2000-11-30  55.943333  51.533333  58.086667  56.320000    ...      18.950000   
2001-11-30  59.332258  56.748387  60.958065  60.096774    ...      20.190000   
2002-11-30  63.560000  57.090000  62.900000  62.066667    ...      20.450000   
2003-11-30  56.816667  54.073333  60.600000  58.400000    ...      17.950000   
2004-11-30  56.273333  52.243333  60.093333  57.520000    ...      17.180000   
2005-11-30  62.729032  57.867742  62.780645  61.806452    ...      16.966667   
2006-11-30  63.193333  58.270000  62.933333  62.350000    ...      17.530000   
2007-11-30  60.876667  56.193333  60.893333  60.430000    ...      19.000000   
2008-11-30  64.703333  61.393333  63.900000  63.110000    ...      22.016667   
2009-11-30  61.8

In [217]:
normalized = all_data.subtract(all_data.mean())
normalized.head()

TEMP                                                    \
STN           690150    722810    722860    722868    722880    722885   
1990-11-30  3.100607 -1.338221  2.580224  3.197686  1.564538  3.802907   
1991-11-30  2.383940  0.484571  3.563557  2.531019  1.791205 -0.177093   
1992-11-30  1.726798 -1.564147  0.490224 -2.802314  1.177871  6.329574   
1993-11-30 -2.688473 -0.494814  1.107213 -3.092852  0.033355  2.743660   
1994-11-30 -0.980765 -5.293314 -6.193110 -5.212314 -5.962129 -2.430426   

                                                      ...         GUST  \
STN           722886    722897    722900    722903    ...       725905   
1990-11-30  1.847687  0.779223  0.915677  3.657740    ...     0.804588   
1991-11-30  4.864354  3.012557  0.002344  3.931073    ...     1.804588   
1992-11-30  4.517687  2.242557  0.995677  3.811073    ...    -3.895412   
1993-11-30  1.092741  0.643309 -0.235935  2.231396    ...     0.564588   
1994-11-30 -3.592313 -4.700777 -4.924323 -2.982260    ...     0.786406   

                                                                        \
STN           725910    725945    725946    725955    745160    746120   
1990-11-30 -4.003610 -5.253413 -4.697976 -9.595523 -0.242418 -0.225377   
1991-11-30  2.155120 -2.136746  0.739524 -0.012190  2.519121 -0.758711   
1992-11-30 -7.759166 -3.036746 -2.097976 -3.835523  0.800439 -6.658711   
1993-11-30  0.055120 -1.786746 -1.106309 -4.139968  2.807582 -2.570832   
1994-11-30 -0.359166 -2.206746  2.463563  0.771143  2.007582  1.107956   

                                          
STN           747185    747187    747188  
1990-11-30  3.313216 -0.085760  6.363059  
1991-11-30  4.856550 -0.214331  1.423377  
1992-11-30 -0.451784 -1.848617  1.811948  
1993-11-30 -1.943450 -0.408617  3.618615  
1994-11-30  3.106550 -1.228617  0.871948  

[5 rows x 236 columns]

In [218]:
normalized.to_csv('../data/preprocessed/CA/CA_normailzed.csv')

In [220]:
normalized = pd.read_csv('../data/preprocessed/CA/CA_normailzed.csv', header=[0,1], index_col=0)
normalized.index = pd.to_datetime(all_data.index)

In [222]:
from sklearn.decomposition import PCA

In [223]:
pca = PCA(n_components=2)

In [224]:
projected = pca.fit_transform(all_data)

In [225]:
projected

array([[-16.2863479 ,  15.38683783],
       [-19.01103516,   1.62081407],
       [-11.9656004 ,   3.90623552],
       [-10.32247717,  11.64793821],
       [ 44.15391401,   2.40850984],
       [-39.60707976, -15.3631593 ],
       [  7.53380163, -33.92077693],
       [ -2.32123077, -36.75706812],
       [ 22.5628509 , -11.53918644],
       [-11.30304166,  -9.17453083],
       [ 33.75815639,  23.35913647],
       [  8.73786927, -21.5166402 ],
       [-12.4812293 ,   7.03955125],
       [ 30.96081181,   4.09879887],
       [ 28.70129352,   6.74771668],
       [-16.72690352,  11.33595954],
       [  1.94525686, -12.86867233],
       [-13.43604352,   4.7129446 ],
       [-13.02382998, -10.29422183],
       [ -7.89077394,  22.242925  ],
       [ 16.06151912,   5.52859257],
       [ 24.94422918,   5.44478777],
       [ -3.99464082, -12.31982947],
       [-17.84412469,  29.92057491],
       [-23.14534412,   8.35276233]])

In [226]:
from plotly import plotly as ply
from plotly import graph_objs as go

In [227]:
trace0 = go.Scatter(
    x=projected[:,0],
    y=projected[:,1],
    name='Projected',
    mode='markers+text',
    text=all_data.index.year,
)
layout = go.Layout(
    title="California transformed PCA"
)
figure = go.Figure(data=[trace0], layout=layout)
ply.iplot(figure, filename='WeatherData/sandiego_PCA')

In [214]:
ply.image.save_as(figure, '../figures/California transformed PCA.png')